In [46]:
import requests
from bs4 import BeautifulSoup
import csv
import os
import itertools
import time
import random
import urllib.robotparser

#Revisant robots.txt es veu sobre quines pagines no es pot fer scrapping. Es crea una funció per a comprobar si es permet
def robot(url_robot, url):
    rp = urllib.robotparser.RobotFileParser()
    rp.set_url(url_robot)
    rp.read()
    user_agent = 'wswp'
    return print('Available:', rp.can_fetch(user_agent, url))

url = 'https://www.amazon.es/gp/bestsellers/books?pf_rd_r=3SGMV0M1FJ0N0CB7MQC0&pf_rd_p=d3259a28-7b77-56ff-ade0-8a02db82407a&pd_rd_r=0c54ad8c-b4d5-4d1a-a3bb-3246e048bdc2&pd_rd_w=m1O3G&pd_rd_wg=ES5Zj&ref_=pd_gw_ri'
url_robot = 'http://www.amazon.com/robots.txt'

robot(url_robot, url)

#S'utilitzen certs parametres de headers per a que el host no detecti que no s'esta utilizant un buscador
headers = {
    'authority': 'www.amazon.es',
    'accept': 'text/html,*/*',
    'sec-fetch-dest': 'empty',
    'x-requested-with': 'XMLHttpRequest',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36',
    'content-type': 'application/x-www-form-urlencoded',
    'origin': 'https://www.amazon.es',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'referer': 'https://www.amazon.es/Aprendiendo-repasar-actividades-infantiles-preescolar/dp/164608070X/ref=zg_bs_books_1?_encoding=UTF8&psc=1&refRID=56ZR35YMVJ6JCAVDV35P',
    'accept-language': 'es-419,es;q=0.9,ca;q=0.8,en;q=0.7',
    'cookie': 'session-id=261-6732941-4022622; i18n-prefs=EUR; csm-hit=tb:s-70XB9ADC3KQKXZNWJ0H7|1586340018542&t:1586340018543; ubid-acbes=261-5214057-4062537; session-id-time=2082758401l; x-wl-uid=1496UkvL3TN/WMSFXNQFQ1TustQRy4vYymz7YCTDp4ebwWWg/9S+j8ZRC8pSw4QG3oYYfusYOZeU=; session-token=wLXXRXA0YpGTaAacKe1aHCqbepbrHvK6d2g816+i0fxdM35bmBFcV0u8dGiDyDasRP9SlGhyBNmO2x8VLgBdBnMFCqOUPYkF5RBy8HBlFkgdFFLyZYcaNJATcd2g36Zac81CZyM0qXovTLe0gc+49/jJdCkOCBDkPft5kXi4lmgAvpWOPSY1Z4hCM30Q2AvbyNhhTOKQ7bOfMsxeAAS4j6QvKruki1g51APumKIwkCJ9J9W21CIu14EyuheLO2Pv',
}

#Es crea una funció per detectar si l'atribut no existeix. En cas de que no existeixi es deixara com una cadena buida: 
def trobar_valor(contingut,identificador):
    
    try:
        caracteristica = contingut.find(class_= identificador).string
    except:
        caracteristica = ""
    return caracteristica

def scrapp(url, headers = headers):
    
    page = requests.get(url, headers)
    soup = BeautifulSoup(page.content)
    return soup
    
        
#Es declara la variable on s'enmagatzemaran els resultats
llista_llibres = []
llista_reviews = []

# Es crea un diccionari per les dades amb el detall de cada llibre
diccionari_atribut = {"Nom": "",
                      "Autor": "",
                      "Puntuacio": "",
                      "Num_resenyes": "",
                      "Categoria": "",
                      "Preu": "",
                      "Paginas": "",
                      "Editor": "",
                      "Idioma": "",
                      "ISBN-10": "",
                      "ISBN-13": "",
                      "Coleccion": "",
                      "Edad recomendada": ""
                     }
# Es crea un diccionari per les dades amb el detall de les reviews
diccionari_reviews = {"Nom_llibre":"",
                      "Usuari": "",
                      "Puntuacio": "",
                      "Titol": "",
                      "Data": "",
                      "Text": "",
                      "Votacions": ""
                     }

#Es crea l'scrapp
#El llistat de llibres de Amazon es divideix en dues pagines, pel que hem de llegir aquestes dues.
for pagina in range(1,3):
    url = "https://www.amazon.es/gp/bestsellers/books/ref=zg_bs_pg_1/259-0380097-5900722?ie=UTF8&pg=%d" % pagina
    soup = scrapp(url)

    # Guardem en la variable llistat el contingut de tots els llibres
    llistat = soup.find(class_="a-ordered-list a-vertical")

    # Recorrem llibre a llibre (aok-inline-block zg-item) buscant les característiques de cada un d'ells. 
    for llib in llistat.find_all('span', class_="aok-inline-block zg-item"):
        
        # Agafem les dades i les emmagatzemem al diccionari
        diccionari_atribut["Nom"] = trobar_valor(llib,"p13n-sc-truncate").strip()
        diccionari_atribut["Autor"] = trobar_valor(llib,"a-row a-size-small")
        diccionari_atribut["Puntuacio"] = trobar_valor(llib,"a-icon-alt")
        diccionari_atribut["Num_resenyes"] = trobar_valor(llib,"a-size-small a-link-normal")
        diccionari_atribut["Categoria"] = trobar_valor(llib,"a-size-small a-color-secondary")
        diccionari_atribut["Preu"] = trobar_valor(llib,"p13n-sc-price")
        
        #Dintre de cada llibre s'accedeix a la pagina del producte per a obtenir més informació
        link = llib.find(class_="a-link-normal").get('href')
        link = "https://www.amazon.es" + str(link)
        print(link)
        
        # Esperem un temps entre peticions per no saturar
        time.sleep(random.random())
        page2 = requests.get(link, headers = headers)
        soup2 = BeautifulSoup(page2.content,'lxml')
        
        #Es selecciona la taula d'on es vol treure informació, amb els detalls del llibre
        taula = soup2.find(id ="detail_bullets_id")
        # Recorrem cada element de la taula de detalls
        for element in taula.find_all('li'):
            text = element.get_text()
            
            # Al text s'utilitzen punts en el codi html per separar elements
            posicio_punts = text.find(":")
            if text[:posicio_punts] in ['Editor', 'Idioma', 'ISBN-10', 'ISBN-13', 'Coleccion', 'Edad recomendada']:
                 diccionari_atribut[text[:posicio_punts]] = text[posicio_punts + 2:]
            elif text[:4] == 'Tapa':
                diccionari_atribut['Paginas'] = text[posicio_punts + 2:]
        
        #  Per a cada llibre, recorrem les reviews més rellevants. Posem try-except per si no té reviews
        revs = soup2.find(class_ = "a-section a-spacing-extra-large reviews-content filterable-reviews-content celwidget")
        try:
            for rev in revs.find_all('div',"a-section review aok-relative"):
                diccionari_reviews['Nom_llibre'] =  diccionari_atribut["Nom"]
                diccionari_reviews["Usuari"] = trobar_valor(rev,"a-profile-name")
                diccionari_reviews["Puntuacio"] = trobar_valor(rev,"a-icon-alt")
                diccionari_reviews["Titol"] = rev.find(class_= 'a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold').find('span').get_text()
                diccionari_reviews["Data"] = trobar_valor(rev,"a-size-base a-color-secondary review-date")
                diccionari_reviews["Text"] = rev.find(class_= 'a-expander-content reviewText review-text-content a-expander-partial-collapse-content').find('span').get_text()
                diccionari_reviews["Votacions"] = trobar_valor(rev,"a-size-base a-color-tertiary cr-vote-text")

                diccionari_reviews = diccionari_reviews

                #Copiem les característiques de cada review en una línea del CSV final, utilitzant "|" com a separador
                linea_rev = list(diccionari_reviews.values())
                linea_rev = "|".join(linea_rev)
                llista_reviews.append(linea_rev)
                
                #Es buida el diccionari per si hi ha algun atribut que no tingui cap valor en la seguent iteració
                diccionari_reviews = diccionari_reviews.fromkeys(diccionari_reviews, "")
        except:
            print("Aquest llibre no té reviews")
            
        #Copiem les característiques de cada llibre en una línea del CSV final, utilitzant "|" com a separador
        #print(diccionari_atribut)
        linea = list(diccionari_atribut.values())
        linea = "|".join(linea)
        llista_llibres.append(linea)
                
        #Es buida el diccionari per si hi ha algun atribut que no tingui cap valor en la seguent iteració
        diccionari_atribut = diccionari_atribut.fromkeys(diccionari_atribut, "")
             
# Escrivim un fitxer csv amb les dades dels llibres més venuts d'amazon
with open('llibres.csv', 'w', newline='') as csvFile:
    csvFile.write("Nom|Autor|Puntuacio|Num_resenyes|Categoria|Preu|Paginas|Editor|Idioma|ISBN-10|ISBN-13|Coleccion|Edad recomendada\n")
    for llibre in llista_llibres:
        csvFile.write(llibre)
        csvFile.write('\n')
        
# Escrivim un fitxer csv amb les reviews dels llibres més venuts d'amazon
with open('reviews.csv', 'w', newline='') as csvFile:
    csvFile.write("Nom_llibre|Usuari|Puntuacio|Titol|Data|Text|Votacions\n")
    for review in llista_reviews:
        csvFile.write(review)
        csvFile.write('\n')



Available: True
https://www.amazon.es/Aprendiendo-repasar-actividades-infantiles-preescolar/dp/164608070X?_encoding=UTF8&psc=1
https://www.amazon.es/chica-nieve-Javier-Castillo/dp/8491292667?_encoding=UTF8&psc=1
https://www.amazon.es/Libro-colorear-para-adultos-Pensamientos/dp/1729502148?_encoding=UTF8&psc=1
https://www.amazon.es/caso-castillo-encantado-Serie-BuscaPistas/dp/8415580703?_encoding=UTF8&psc=1
https://www.amazon.es/Reina-roja-Trama-Juan-G%C3%B3mez-Jurado/dp/8466664416?_encoding=UTF8&psc=1
https://www.amazon.es/Mandalas-Para-Meditar-inspirador-relajaci%C3%B3n/dp/1533680000?_encoding=UTF8&psc=1
https://www.amazon.es/Caligraf%C3%ADa-Escritura-Min%C3%BAsculas-Iniciaci%C3%B3n-May%C3%BAsculas/dp/8485109279?_encoding=UTF8&psc=1
https://www.amazon.es/Sapiens-animales-dioses-historia-humanidad/dp/8499926223?_encoding=UTF8&psc=1
https://www.amazon.es/cuento-perfecto-Femenino-singular/dp/8491291911?_encoding=UTF8&psc=1
https://www.amazon.es/Amor-Colorear-antiestr%C3%A9s-adolescentes-r

https://www.amazon.es/madre-Frankenstein-Andanzas-Almudena-Grandes/dp/8490667802?_encoding=UTF8&psc=1
https://www.amazon.es/Principito-Antoine-Saint-Exup%C3%A9ry-Saint-Exupery/dp/8498381495?_encoding=UTF8&psc=1
https://www.amazon.es/Harry-Potter-Piedra-Filosofal-Rowling/dp/8478884459?_encoding=UTF8&psc=1
https://www.amazon.es/Isadora-Moon-mete-l%C3%ADo-Infantil/dp/8420486329?_encoding=UTF8&psc=1
https://www.amazon.es/Los-Futbol%C3%ADsimos-El-misterio-%C3%A1rbitros-dormidos/dp/8467561351?_encoding=UTF8&psc=1
https://www.amazon.es/100-Animal-Mandalas-Para-Colorear/dp/B085RSFCKK?_encoding=UTF8&psc=1
https://www.amazon.es/Mandalas-Complejos-Libro-Colorear-Adultos/dp/1535004150?_encoding=UTF8&psc=1
https://www.amazon.es/Aprender-escribir-n%C3%BAmeros-infantiles-matem%C3%A0ticas/dp/1695493826?_encoding=UTF8&psc=1
https://www.amazon.es/Los-Futbol%C3%ADsimos-El-misterio-robo-imposible/dp/8467574151?_encoding=UTF8&psc=1
https://www.amazon.es/Mi-cuaderno-actividades-a%C3%B1os-entretenimiento/dp/